In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np
import os
import time
import copy
from tqdm.auto import tqdm

In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# --- Cấu hình chung ---
DATA_DIR = '/content/drive/MyDrive/Data/VinaFood21/VinaFood21'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

if not os.path.exists(DATA_DIR):
    print(f"LỖI: Không tìm thấy thư mục '{DATA_DIR}'.")
    print("Vui lòng kiểm tra lại đường dẫn Google Drive của bạn.")

NUM_CLASSES = 21 # VinaFood21

Sử dụng thiết bị: cuda:0


In [ ]:
def get_dataloaders(img_size, batch_size=32):
    """Tải dữ liệu với kích thước ảnh cụ thể."""
    print(f"\nĐang tải dữ liệu với kích thước {img_size}x{img_size}...")
    data_transforms = {
        'train': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    try:
        image_datasets = {x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
                          for x in ['train', 'test']}
        dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=(x=='train'), num_workers=2)
                       for x in ['train', 'test']}
        dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
        class_names = image_datasets['train'].classes
        print(f"Tải xong. {dataset_sizes['train']} ảnh train, {dataset_sizes['test']} ảnh test.")
        return dataloaders, dataset_sizes, class_names
    except FileNotFoundError:
        print(f"Lỗi: Không tìm thấy thư mục 'train' hoặc 'test' trong {DATA_DIR}")
        return None, None, None

In [ ]:
def quick_evaluate(model, dataloader):
    """Hàm đánh giá nhanh (chỉ tính Acc) để xem sau mỗi epoch."""
    model.eval()
    running_corrects = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
    acc = running_corrects.double() / total
    print(f'Test Acc (Validation): {acc:.4f}')
    model.train() # Chuyển lại chế độ train

In [ ]:
def train_model_torchvision(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs):
    """Hàm huấn luyện cho LeNet và ResNet (output chuẩn)."""
    since = time.time()
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train() # Đặt mô hình ở chế độ training
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], desc=f"Training Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Đánh giá nhanh trên tập test (validation) sau mỗi epoch
        quick_evaluate(model, dataloaders['test'])


    time_elapsed = time.time() - since
    print(f'\nHuấn luyện hoàn tất trong {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

In [ ]:
def train_model_googlenet(model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs):
    """Hàm huấn luyện đặc biệt cho GoogLeNet (có 3 output)."""
    since = time.time()
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train() # Đặt mô hình ở chế độ training
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders['train'], desc=f"Training Epoch {epoch+1}"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                main_out, aux1_out, aux2_out = model(inputs)
                loss_main = criterion(main_out, labels)
                loss_aux1 = criterion(aux1_out, labels)
                loss_aux2 = criterion(aux2_out, labels)
                loss = loss_main + 0.3 * (loss_aux1 + loss_aux2)
                _, preds = torch.max(main_out, 1)
                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc = running_corrects.double() / dataset_sizes['train']
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Đánh giá nhanh trên tập test (validation) sau mỗi epoch
        quick_evaluate(model, dataloaders['test'])

    time_elapsed = time.time() - since
    print(f'\nHuấn luyện hoàn tất trong {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model

In [ ]:
def evaluate_model_metrics(model, dataloader):
    """Đánh giá mô hình trên tập test và tính Precision, Recall, F1-macro."""
    print("\nĐang đánh giá chi tiết trên tập Test...")
    model = model.to(device)
    model.eval() # Chuyển sang chế độ eval

    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels,
        all_preds,
        average='macro',
        zero_division=0
    )
    accuracy = accuracy_score(all_labels, all_preds)

    print("\n--- KẾT QUẢ ĐÁNH GIÁ (Macro Average) ---")
    print(f'Accuracy:  {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall:    {recall:.4f}')
    print(f'F1-Score:  {f1:.4f}')
    print("-" * 39)

In [ ]:
# --- Tải Dữ Liệu ---
dataloaders_28, sizes_28, classes_28 = get_dataloaders(img_size=28, batch_size=64)
dataloaders_224, sizes_224, classes_224 = get_dataloaders(img_size=224, batch_size=32)


Đang tải dữ liệu với kích thước 28x28...
Tải xong. 10089 ảnh train, 6693 ảnh test.

Đang tải dữ liệu với kích thước 224x224...
Tải xong. 10089 ảnh train, 6693 ảnh test.


In [ ]:
# -------------------------------------------------------------------
# BÀI 1: LeNet (28x28)
# -------------------------------------------------------------------
print("\n" + "="*50)
print("BẮT ĐẦU BÀI 1: LeNet")
print("="*50)

class LeNet(nn.Module):
    def __init__(self, num_classes=21):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, padding=2)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

if dataloaders_28:
    model_lenet = LeNet(num_classes=NUM_CLASSES).to(device)
    criterion_lenet = nn.CrossEntropyLoss()
    optimizer_lenet = optim.Adam(model_lenet.parameters(), lr=0.001)

    NUM_EPOCHS_B1 = 30 # Số epochs hợp lý cho LeNet
    print(f"Huấn luyện LeNet trong {NUM_EPOCHS_B1} epochs...")

    model_lenet_trained = train_model_torchvision(
        model_lenet, dataloaders_28, sizes_28,
        criterion_lenet, optimizer_lenet, num_epochs=NUM_EPOCHS_B1
    )

    print("\n[Bài 1: LeNet] Đánh giá cuối cùng:")
    evaluate_model_metrics(model_lenet_trained, dataloaders_28['test'])
else:
    print("Bỏ qua Bài 1 do không tải được dữ liệu.")


BẮT ĐẦU BÀI 1: LeNet
Huấn luyện LeNet trong 30 epochs...
Epoch 1/30
----------


Training Epoch 1:   0%|          | 0/158 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss: 2.8627 Acc: 0.1311


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e93b4be45e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e93b4be45e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^Exception ignored in: ^<function _MultiProcessingDataLoaderIter.__del__ at 0x7e93b4be45e0>^
^Traceback (most recent call last):
^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
^    ^^self._shutdown_workers()^
^^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
^    
if w.is_alive():  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive

     assert self._parent_pid == os.getpid(), 'can only test a child process' 
          ^ ^ ^ ^ ^ ^ ^^^^^^^^^^^^
  File

Test Acc (Validation): 0.2716

Huấn luyện hoàn tất trong 161m 36s

[Bài 1: LeNet] Đánh giá cuối cùng:

Đang đánh giá chi tiết trên tập Test...


Evaluating:   0%|          | 0/105 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e93b4be45e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e93b4be45e0>  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
<function _MultiProcessingDataLoaderIter.__del__ at 0x7e93b4be45e0>
    
Traceback (most recent call last):
Traceback (most recent call last):
if w.is_alive():  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__

         self._shutdown_workers()self._shutdown_workers() 

   File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, 


--- KẾT QUẢ ĐÁNH GIÁ (Macro Average) ---
Accuracy:  0.2716
Precision: 0.2698
Recall:    0.2614
F1-Score:  0.2567
---------------------------------------
